In [1]:
# Import matplotlib.pyplot
from random import random

import matplotlib.pyplot as plt
# Import pandas
import pandas as pd
# Import geopandas
import geopandas as gpd
# import Folium
import folium
import shapely.geometry

In [4]:
# Vamos por nuestros datos
df = pd.read_csv('DATASET/recorrido_2.csv')
# Vemos su contenido
df.head()

,Coordenadas,Tipo Transporte,Coordenadas de Accidente,Tipo Accidente,Tiempo,Costo,Descripcion Accidente
0,"(19.383082625228962, -99.1906271466764)",CAMINAR,Ninguna,Ninguno,05:30,$0.00,NaN
1,"(19.38367013560702, -99.19071438138118)",CAMINAR,Ninguna,Ninguno,05:33,$0.00,NaN
2,"(19.382961076037986, -99.18416246474426)",CAMINAR,Ninguna,Ninguno,05:40,$0.00,NaN
3,"(19.38377832377402, -99.18375208678239)",CAMINAR,Ninguna,Ninguno,05:43,$0.00,NaN
4,"(19.382705527130742, -99.18137564967644)",CAMINAR,Ninguna,Ninguno,05:48,$0.00,NaN


In [5]:
# Vemos su descripcion
df.info()

# Vemos su tamaño mxn , m = filas, n = columnas
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Coordenadas               30 non-null     object
 1   Tipo Transporte           30 non-null     object
 2   Coordenadas de Accidente  30 non-null     object
 3   Tipo Accidente            30 non-null     object
 4   Tiempo                    30 non-null     object
 5   Costo                     30 non-null     object
 6   Descripcion Accidente     1 non-null      object
dtypes: object(7)
memory usage: 1.8+ KB


(30, 7)

In [6]:
# Acomodamos los datos
df.Coordenadas =  df.Coordenadas.str.strip('()')
df[['Latitude', 'Longitude']] = df.Coordenadas.str.split(',', expand=True)
df.Costo =  df.Costo.str.strip('$')
# Casteamos los datos al tipo de dato
df = df.astype({'Longitude': float, 'Latitude': float, 'Costo': float})

In [7]:
# Eliminamos la columna Coordenadas
df = df.drop('Coordenadas', axis=1)
df = df.drop('Coordenadas de Accidente', axis=1)
df = df.drop('Tipo Accidente', axis=1)
df = df.drop('Descripcion Accidente', axis=1)
# El resultado
df.head()

,Tipo Transporte,Tiempo,Costo,Latitude,Longitude
0,CAMINAR,05:30,0.0,19.383083,-99.190627
1,CAMINAR,05:33,0.0,19.383670,-99.190714
2,CAMINAR,05:40,0.0,19.382961,-99.184162
3,CAMINAR,05:43,0.0,19.383778,-99.183752
4,CAMINAR,05:48,0.0,19.382706,-99.181376


In [8]:
df.info()

df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Tipo Transporte  30 non-null     object 
 1   Tiempo           30 non-null     object 
 2   Costo            30 non-null     float64
 3   Latitude         30 non-null     float64
 4   Longitude        30 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.3+ KB


(30, 5)

In [9]:
# Pasamos la informacion a un GeoDataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs ="EPSG:4326")

gdf['loc'] = gdf.geometry.apply(lambda x:shapely.geometry.mapping(x))
# Show
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Tipo Transporte  30 non-null     object  
 1   Tiempo           30 non-null     object  
 2   Costo            30 non-null     float64 
 3   Latitude         30 non-null     float64 
 4   Longitude        30 non-null     float64 
 5   geometry         30 non-null     geometry
 6   loc              30 non-null     object  
dtypes: float64(3), geometry(1), object(3)
memory usage: 1.8+ KB


In [10]:
gdf.head()

,Tipo Transporte,Tiempo,Costo,Latitude,Longitude,geometry,loc
0,CAMINAR,05:30,0.0,19.383083,-99.190627,POINT (-99.19063 19.38308),"{'type': 'Point', 'coordinates': (-99.19062714..."
1,CAMINAR,05:33,0.0,19.383670,-99.190714,POINT (-99.19071 19.38367),"{'type': 'Point', 'coordinates': (-99.19071438..."
2,CAMINAR,05:40,0.0,19.382961,-99.184162,POINT (-99.18416 19.38296),"{'type': 'Point', 'coordinates': (-99.18416246..."
3,CAMINAR,05:43,0.0,19.383778,-99.183752,POINT (-99.18375 19.38378),"{'type': 'Point', 'coordinates': (-99.18375208..."
4,CAMINAR,05:48,0.0,19.382706,-99.181376,POINT (-99.18138 19.38271),"{'type': 'Point', 'coordinates': (-99.18137564..."


In [11]:
cdmx = [19.432608, -99.133209]

# Creamos el mapa
m = folium.Map(location=cdmx, zoom_start=12, tiles='cartodbpositron')

In [ ]:
geopath = gdf.geometry.to_json()
#print(geopath)
poligonos = folium.features.GeoJson(geopath)
m.add_child(poligonos)
folium.LayerControl().add_to(m)
m

In [16]:
m = gdf.explore(tiles="CartoDB positron")

m

In [17]:
points = gdf[['Longitude', 'Latitude']].values.tolist()


In [18]:
from numpy import sin, cos, arccos, pi, round

x = points[0]
y = points[1]

def rad2deg(radians):
    degrees = radians * 180 / pi
    return degrees

def deg2rad(degrees):
    radians = degrees * pi / 180
    return radians

def getDistanceBetweenPointsNew(latitude1, longitude1, latitude2, longitude2, unit = 'miles'):

    theta = longitude1 - longitude2

    distance = 60 * 1.1515 * rad2deg(
        arccos(
            (sin(deg2rad(latitude1)) * sin(deg2rad(latitude2))) +
            (cos(deg2rad(latitude1)) * cos(deg2rad(latitude2)) * cos(deg2rad(theta)))
        )
    )

    if unit == 'miles':
        return round(distance, 2)
    if unit == 'kilometers':
        return round(distance * 1.609344, 3)


In [ ]:
distances = []
for i in range(0, len(points)):
    k = i + 1
    if k == len(points):
        print("Se acabo")
    else:
        x = points[i]
        x2 = points[k]
        distances.append(getDistanceBetweenPointsNew(x[0], x[1], x2[0], x2[1], 'kilometers') * 1000)
        # print("[{0}] vs [{1}]".format(points[i], points[k]))

distances
df_distances = pd.DataFrame(distances, columns=['Distancias'])
df_distances

In [20]:
m = gdf.explore(tiles="CartoDB positron")
folium.PolyLine(gdf[['Latitude','Longitude']].values.tolist(), color="purple", weight=2.5, opacity=1).add_to(m)
m